<a href="https://colab.research.google.com/github/DuanMingbai/PyTorch_Tutorials_cn/blob/main/Introduction%20to%20PyTorch/5_%E5%BB%BA%E7%AB%8B%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#建立神经网络

神经网络由对数据执行操作的层/模块组成。[torch.nn](https://pytorch.org/docs/stable/nn.html)命名空间为你构建自己的神经网络提供了一整套构建模块，在PyTorch中每一个模块都是[nn.Model](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)的子类，神经网络本身就是一个模块本身，由其他模块（层）组成。这种嵌套的结构允许更方便的去管理和建造更复杂的神经网络架构。

\\
在本节中，我们会构建一个神经网络来对FasionMINST中的图片进行分类。

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

##获取训练设备

如果可能的话，我们希望我们的模型可以运行在类似GPU这种加速设备上，让我们检查一下torch.cuda是否可用，或者我们继续去使用CPU。

In [ ]:
device = "cuda" if torch.cuda.is_available() else "CPU"
print(f"Using {device} device")

Using cuda device


##定义类

我们通过nn.Module的子类定义我们自己的神经网络，用```__init__```函数初始化神经网络层数，每个 ```nn.Module```子类都在```forward```方法中实现对输入数据的操作。

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建一个NeuralNetwork的实例，并且把它挪到GPU(```device```)上运行，然后打印。

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


要使用的模型来处理我们将输入的数据。这样会执行模型的```forward```函数，以及后台的一些操作。不要直接调用```model.forward()```！！！！！！

在输入的时候调用模型会返回一个dim=0二维的tensor,第一维dim=0对应了输出的十个原始的预测值，第二维dim=1对应了每一个输出的单个值，我们将它传递到一个```nn.Module```模块的实例来获得预测的可能性。

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')



\\


---

\\

##模型层

让我们来分解一下FashionMINST模型的层数，我们会定义一个包含3张大小为28×28的图片的样本，然后观察一下，我们将他们送进神经网络后会发生什么。

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


##nn.Flatten

我们初始化nn.Flatten层，去将一个大小为28×28的2D图片转换成大小为（28的平方）784像素组值的连续数组（在dim=0的时候保持小批量维度）

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


##nn.Linear

线性层也是一个模块，用它存储的权重（w）和偏差（b）来对输入的数据进行现行转换。

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


##nn.ReLU

非线性激活函数创建了一个从模型的输入到输出的复杂映射。他们在线性变换之后引入非线性，帮助神经网络学习各种各样的现象。

\\
在这个模型中我们在线性层中使用[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)，你还可以在你的模型中引入其他其他非线性模型。

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.4221, 0.0799, 0.2920, 0.0000, 0.3960, 0.0000, 0.4093, 0.0000, 0.0000,
         0.2008, 0.3254, 0.0000, 0.0000, 0.0027, 0.0193, 0.0000, 0.0000, 0.0000,
         0.3248, 0.0228],
        [0.2284, 0.1962, 0.2006, 0.2729, 0.4946, 0.0866, 0.0936, 0.0000, 0.0000,
         0.1356, 0.0692, 0.0000, 0.0000, 0.4252, 0.0000, 0.1199, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.6752, 0.2179, 0.0781, 0.0000, 0.6522, 0.0000, 0.5320, 0.0000, 0.0647,
         0.0000, 0.4191, 0.0359, 0.0000, 0.0000, 0.0000, 0.1831, 0.0000, 0.0000,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.4221, 0.0799, 0.2920, 0.0000, 0.3960, 0.0000, 0.4093, 0.0000, 0.0000,
         0.2008, 0.3254, 0.0000, 0.0000, 0.0027, 0.0193, 0.0000, 0.0000, 0.0000,
         0.3248, 0.0228],
        [0.2284, 0.1962, 0.2006, 0.2729, 0.4946, 0.0866, 0.0936, 0.0000, 0.0000,
         0.1356, 0.0692, 0.0000, 0.0000, 0.4252, 0.0000, 0.1199, 0.0000, 0.0000,
         0.0000, 0.0000],
       

##nn.Sequential

nn.Sequential是一个有序列的模块，数据按照定义的顺序通过所有模块，你可以使用序列容器搭建一个快速网络，例如```seq_modules```。

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

##nn.Softmax

神经网络最后一个线性层返回一个逻辑值，-原来的值在[-∞,+∞]-原来的值被送入nn.Softmax模块，逻辑值就被缩放到[0, 1]之间表示模型对每个类别预测的可能性。```dim```参数的意思是，在这个纬度里面的值的和必须是1。

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

##模型参数

许多层（layer）在神经网络中是参数化的，也就是说在训练的过程中会更新相关的权重。```nn.Module```的子类会自动跟踪你模型中的所有子类，使用你的模型的```parameters()```或者```named_parameters()```使所有的参数都可以被访问。

\\
在这个例子中，我们遍历每一个参数，并且预览它的值。

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0097, -0.0246, -0.0340,  ..., -0.0133, -0.0165,  0.0280],
        [ 0.0236,  0.0041, -0.0156,  ..., -0.0140, -0.0224,  0.0296]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0128, -0.0032], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0192, -0.0031, -0.0141,  ..., -0.0266, -0.0119, -0.0172],
        [ 0.0401,  0.0261, -0.0388,  ..., -0.0308,  0.0155,  0.0024]],
       device='cuda:0', grad_fn=<Sl

##阅读更多

[torch.nn API](https://pytorch.org/docs/stable/nn.html)